In [1]:
# Import required Libraries

import pandas as pd
import os
from pandas import json_normalize
import json
import requests
from IPython.display import JSON

In [81]:
# Weather API Fetch Function

def world_weather_fetch(query):
    
        # Set Key
        weather_key = os.environ['weather_api_key']
        
        # Set Parameters for API
        weather_url = "http://api.worldweatheronline.com/premium/v1/weather.ashx?"
        location = query
        weather_response = requests.get(weather_url,params={
                                                  'q':location,
                                                  'date':'2019-01-01',
                                                  'fx':'no',
                                                  'cc':'no',
                                                  'mca':'yes',
                                                  'format':'json',
                                                  'key':weather_key})
        
        # Call API and direct returned information
        weather_result = weather_response.json()
        df_weather = json_normalize(weather_result)
        return weather_result, df_weather

In [111]:
# Function to Call Weather API Function through loop and then collect/compile data

def compile_weather_data(string_list):
    
    # Set up initial dataframe to add column datas onto
    init_result,init_data = world_weather_fetch(string_list[0])
    init_data = json_normalize(data=init_data['data.ClimateAverages'][0][0]['month'])
    base_data = init_data[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']].apply(pd.to_numeric)
    
    # Loop through list of locations and collect API data
    for i in string_list[1:]:
        result,dataframe = world_weather_fetch(i)
        weather_dataframe = json_normalize(data=dataframe['data.ClimateAverages'][0][0]['month'])
        base_data[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']] = base_data + weather_dataframe[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']].apply(pd.to_numeric)
    
    # Calculate Average of each column value based on each city added
    base_data = base_data/len(string_list)
    # Replace column data with adjusted values
    init_data[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']] = init_data[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']].replace(base_data[['avgRainDays','avgDryDays','avgCloud','avgSnowDays']])
    return init_data
        

In [83]:
# Import Data list for Weather API Queries
import pickle
infile = open('most_freq_delayed_flight_locations_2019','rb')
most_frequently_delayed = pickle.load(infile)
infile.close()

In [119]:
data = compile_weather_data(most_frequently_delayed)
weather_data_delayed = data[['name','avgRainDays','avgDryDays','avgCloud','avgSnowDays']]